<a href="https://colab.research.google.com/github/Ahmad162539/Smartbuilding/blob/main/1_Experiments_for_University_Dormitory_Building_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load data

In [48]:
import pandas as pd

url_univdorm= 'https://raw.githubusercontent.com/irenekarijadi/RF-LSTM-CEEMDAN/main/Dataset/data%20of%20UnivDorm_Prince.csv'
univdorm= pd.read_csv(url_univdorm)
data_univdorm= univdorm[(univdorm['timestamp'] > '2015-03-01') & (univdorm['timestamp'] < '2015-06-01')]
dfs_univdorm=data_univdorm['energy']
datas_univdorm=pd.DataFrame(dfs_univdorm)


## import libraries

In [49]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

In [50]:
pip install EMD-signal

In [51]:
from PyEMD import CEEMDAN

In [52]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

import numpy
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics

import time

In [53]:
pip install dataframe_image

In [54]:
import dataframe_image as dfi

## Import all functions from another notebook for building prediction methods

In [55]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


#Parameters Settings

max_features=8 # the feature number of each node is set to 8, as it is suggested to be one-third of the feature's number (Lahouar and Slama,2017)
lr=0.001 #learning rate LSTM as recommended in (Kingma and Ba, 2014)
optimizer='Adam' #Adam is used as the optimizer for LSTM as it is computationally efficient (Dubey, Ashutosh Kumar, et al,2021)
neuron=64
epoch=100 #(Jorges et al, 2021)
batch_size=64 #(Kandel et al,2020)
n_hours=24 #in this study, to predict one hour ahead predicton, we use input from the previous 24 hour energy consumption
data_partition=0.8 #in this study, we divided the data into 80% of data as training data and 20% of the data as testing data

In [56]:
import numpy
import pandas as pd
import tensorflow as tf
from PyEMD import CEEMDAN
import warnings
warnings.filterwarnings("ignore")


### import the libraries
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from math import sqrt

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred):
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100



# In[25]:


def lr_model(datass,look_back,data_partition):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()
    import tensorflow as tf

    numpy.random.seed(1234)
    tf.random.set_seed(1234)

    from sklearn.linear_model import LinearRegression

    grid = LinearRegression()

    grid.fit(X,y)
    y_pred_train_lr= grid.predict(X)
    y_pred_test_lr= grid.predict(X1)

    y_pred_train_lr=pd.DataFrame(y_pred_train_lr)
    y_pred_test_lr=pd.DataFrame(y_pred_test_lr)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_lr= sc_y.inverse_transform (y_pred_test_lr)
    y_pred_train1_lr=sc_y.inverse_transform (y_pred_train_lr)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_rf=pd.DataFrame(y_pred_test1_lr)
    y_pred_train1_rf=pd.DataFrame(y_pred_train1_lr)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_lr)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_lr))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_lr)


    return mape,rmse,mae


# In[26]:


def svr_model(datass,look_back,data_partition):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    numpy.random.seed(1234)

    import tensorflow as tf
    tf.random.set_seed(1234)


    from sklearn.svm import SVR

    grid = SVR()
    grid.fit(X,y)
    y_pred_train_svr= grid.predict(X)
    y_pred_test_svr= grid.predict(X1)

    y_pred_train_svr=pd.DataFrame(y_pred_train_svr)
    y_pred_test_svr=pd.DataFrame(y_pred_test_svr)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_svr= sc_y.inverse_transform (y_pred_test_svr)
    y_pred_train1_svr=sc_y.inverse_transform (y_pred_train_svr)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_svr=pd.DataFrame(y_pred_test1_svr)
    y_pred_train1_svr=pd.DataFrame(y_pred_train1_svr)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_svr)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_svr))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_svr)


    return mape,rmse,mae


# In[27]:



def ann_model(datass,look_back,data_partition):


    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    import numpy

    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))


    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)


    from sklearn.neural_network import MLPRegressor

    model= MLPRegressor(random_state=1,activation='tanh').fit(X,y)

    numpy.random.seed(1234)


    # make predictions
    y_pred_train = model.predict(X)
    y_pred_test = model.predict(X1)
    y_pred_test= numpy.array(y_pred_test).ravel()

    y_pred_test=pd.DataFrame(y_pred_test)
    y1=pd.DataFrame(y1)

    y_pred_test1= sc_y.inverse_transform (y_pred_test)
    y_test= sc_y.inverse_transform (y1)







    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1)


    return mape,rmse,mae


# In[28]:


def rf_model(datass,look_back,data_partition,max_features):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    from sklearn.ensemble import RandomForestRegressor

    grid = RandomForestRegressor(max_features=max_features)
    grid.fit(X,y)
    y_pred_train_rf= grid.predict(X)
    y_pred_test_rf= grid.predict(X1)

    y_pred_train_rf=pd.DataFrame(y_pred_train_rf)
    y_pred_test_rf=pd.DataFrame(y_pred_test_rf)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_rf= sc_y.inverse_transform (y_pred_test_rf)
    y_pred_train1_rf=sc_y.inverse_transform (y_pred_train_rf)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_rf=pd.DataFrame(y_pred_test1_rf)
    y_pred_train1_rf=pd.DataFrame(y_pred_train1_rf)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_rf)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_rf))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_rf)


    return mape,rmse,mae


# In[29]:


def lstm_model(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):
    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()
    trainX1 = numpy.reshape(X, (X.shape[0],1,X.shape[1]))
    testX1 = numpy.reshape(X1, (X1.shape[0],1,X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation
    from keras.layers import LSTM


    neuron=neuron
    model = Sequential()
    model.add(LSTM(units = neuron,input_shape=(trainX1.shape[1], trainX1.shape[2])))
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mse',optimizer=optimizer)
#    model.summary()


  # Fitting the RNN to the Training s
    model.fit(trainX1, y, epochs = epoch, batch_size = batch_size,verbose=0)
  # make predictions
    y_pred_train = model.predict(trainX1)
    y_pred_test = model.predict(testX1)
    y_pred_test= numpy.array(y_pred_test).ravel()

    y_pred_test=pd.DataFrame(y_pred_test)
    y_pred_test1= sc_y.inverse_transform (y_pred_test)
    y1=pd.DataFrame(y1)

    y_test= sc_y.inverse_transform (y1)

    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error

    from sklearn import metrics

    mape=mean_absolute_percentage_error(y_test,y_pred_test1)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1)
    return mape,rmse,mae


# In[30]:


###################################################hybrid based ceemdan####################################################
def hybrid_ceemdan_rf(datass,look_back,data_partition,max_features):


    import numpy as np
    import pandas as pd

    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T

    import pandas as pd

    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    for col in data_imf:

        datasetss2=pd.DataFrame(data_imf[col])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from sklearn.ensemble import RandomForestRegressor


        grid = RandomForestRegressor(max_features=max_features)
        grid.fit(X,y)
        y_pred_train= grid.predict(X)
        y_pred_test= grid.predict(X1)

        y_pred_test=pd.DataFrame(y_pred_test)
        y_pred_train=pd.DataFrame(y_pred_train)

        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)

        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)


    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)
    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)




    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)




   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)



    return mape,rmse,mae


# In[31]:


def hybrid_ceemdan_lstm(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):


    from PyEMD import CEEMDAN

    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T

    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    for col in data_imf:

        datasetss2=pd.DataFrame(data_imf[col])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout, Activation
        from keras.layers import LSTM


        neuron=neuron

        model = Sequential()
        model.add(LSTM(units = neuron,input_shape=(trainX.shape[1], trainX.shape[2])))
        model.add(Dense(1))
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mse',optimizer=optimizer)

        numpy.random.seed(1234)


        # Fitting the RNN to the Training set
        model.fit(trainX, y, epochs = epoch, batch_size = batch_size,verbose=0)

        # make predictions
        y_pred_train = model.predict(trainX)
        y_pred_test = model.predict(testX)

        # make predictions

        y_pred_test= numpy.array(y_pred_test).ravel()
        y_pred_test=pd.DataFrame(y_pred_test)
        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)
        y_pred_train= numpy.array(y_pred_train).ravel()
        y_pred_train=pd.DataFrame(y_pred_train)


        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)


    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)


    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)



   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)

    return mape,rmse,mae


# In[32]:


def proposed_method(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):

    from PyEMD import CEEMDAN


    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T



    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    n_imf=len(data_imf.columns)

    k=list(range(1,n_imf))
    m=[0]


    for i in m:

        datasetss2=pd.DataFrame(data_imf[i])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from sklearn.ensemble import RandomForestRegressor


        grid = RandomForestRegressor(max_features=max_features)
        grid.fit(X,y)
        y_pred_train= grid.predict(X)
        y_pred_test= grid.predict(X1)

        y_pred_test=pd.DataFrame(y_pred_test)
        y_pred_train=pd.DataFrame(y_pred_train)

        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)

        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)




    for i in k:

        datasetss2=pd.DataFrame(data_imf[i])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout, Activation
        from keras.layers import LSTM

        neuron=neuron

        model = Sequential()
        model.add(LSTM(units = neuron,input_shape=(trainX.shape[1], trainX.shape[2])))
        model.add(Dense(1))
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mse',optimizer=optimizer)


        numpy.random.seed(1234)


     # Fitting the RNN to the Training set
        model.fit(trainX, y, epochs = epoch, batch_size = batch_size,verbose=0)


        # make predictions
        y_pred_train = model.predict(trainX)
        y_pred_test = model.predict(testX)

        # make predictions


        y_pred_test= numpy.array(y_pred_test).ravel()
        y_pred_test=pd.DataFrame(y_pred_test)
        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)
        y_pred_train= numpy.array(y_pred_train).ravel()
        y_pred_train=pd.DataFrame(y_pred_train)


        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)

    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)
    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)


    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)


    import numpy as np


   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)

    return mape,rmse,mae,a,y_test


## Import parameter settings

In [57]:

max_features=8 # the feature number of each node is set to 8, as it is suggested to be one-third of the feature's number (Lahouar and Slama,2017)
lr=0.001 #learning rate LSTM as recommended in (Kingma and Ba, 2014)
optimizer='Adam' #Adam is used as the optimizer for LSTM as it is computationally efficient (Dubey, Ashutosh Kumar, et al,2021)
neuron=64
epoch=100 #(Jorges et al, 2021)
batch_size=64 #(Kandel et al,2020)
hours=24 #in this study, to predict one hour ahead predicton, we use input from the previous 24 hour energy consumption
data_partition=0.8 #in this study, we divided the data into 80% of data as training data and 20% of the data as testing data

## Run the experiments
### Run this following cell will train and test the proposed method and other benchmark methods on University Dormitory Dataset

In [58]:
pip install keras

In [59]:
from keras.layers import LSTM

In [60]:
#Linear Regression

start_time = time.time()
lr_univdorm=lr_model(datas_univdorm,hours,data_partition)
lr_time_univdorm=time.time() - start_time
print("--- %s seconds - Linear Regression- univdorm ---" % (lr_time_univdorm))

#Support Vector Regression
start_time = time.time()
svr_univdorm=svr_model(datas_univdorm,hours,data_partition)
svr_time_univdorm=time.time() - start_time
print("--- %s seconds - Support Vector Regression- univdorm ---" % (svr_time_univdorm))


#ANN
start_time = time.time()
ann_univdorm=ann_model(datas_univdorm,hours,data_partition)
ann_time_univdorm=time.time() - start_time
print("--- %s seconds - ANN- univdorm ---" % (ann_time_univdorm))

#random forest
start_time = time.time()
rf_univdorm=rf_model(datas_univdorm,hours,data_partition,max_features)
rf_time_univdorm=time.time() - start_time
print("--- %s seconds - Random Forest- univdorm ---" % (rf_time_univdorm))

#LSTM
start_time = time.time()
lstm_univdorm=lstm_model(datas_univdorm,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
lstm_time_univdorm=time.time() - start_time
print("--- %s seconds - lstm- univdorm ---" % (lstm_time_univdorm))


#CEEMDAN RF
start_time = time.time()
ceemdan_rf_univdorm=hybrid_ceemdan_rf(dfs_univdorm,hours,data_partition,max_features)
ceemdan_rf_time_univdorm=time.time() - start_time
print("--- %s seconds - ceemdan_rf- univdorm ---" % (ceemdan_rf_time_univdorm))

#CEEMDAN LSTM
start_time = time.time()
ceemdan_lstm_univdorm=hybrid_ceemdan_lstm(dfs_univdorm,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
ceemdan_lstm_time_univdorm=time.time() - start_time
print("--- %s seconds - ceemdan_lstm- univdorm ---" % (ceemdan_lstm_time_univdorm))


#proposed method
start_time = time.time()
proposed_method_univdorm=proposed_method(dfs_univdorm,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
proposed_method_time_univdorm=time.time() - start_time
print("--- %s seconds - proposed_method- univdorm ---" % (proposed_method_time_univdorm))



--- 0.13720488548278809 seconds - Linear Regression- univdorm ---
--- 1.2177515029907227 seconds - Support Vector Regression- univdorm ---
--- 4.076772451400757 seconds - ANN- univdorm ---
--- 1.3644065856933594 seconds - Random Forest- univdorm ---
14/14 [==============================] - 0s 2ms/step
--- 12.78627634048462 seconds - lstm- univdorm ---
--- 46.89029669761658 seconds - ceemdan_rf- univdorm ---
14/14 [==============================] - 0s 2ms/step
--- 141.1579830646515 seconds - ceemdan_lstm- univdorm ---
14/14 [==============================] - 0s 2ms/step
--- 132.06445050239563 seconds - proposed_method- univdorm ---


## Summarize of experimental results with running time
### Run this following cell will summarize the result and generate output used in Section 4.4 (Table 3) for University Dormitory dataset

In [61]:
running_time_univdorm=pd.DataFrame([lr_time_univdorm,svr_time_univdorm,ann_time_univdorm,
                                   rf_time_univdorm,lstm_time_univdorm,ceemdan_rf_time_univdorm,
                                   ceemdan_lstm_time_univdorm,proposed_method_time_univdorm])
running_time_univdorm=running_time_univdorm.T
running_time_univdorm.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
proposed_method_univdorm_df=proposed_method_univdorm[0:3]
result_univdorm=pd.DataFrame([lr_univdorm,svr_univdorm,ann_univdorm,rf_univdorm,lstm_univdorm,ceemdan_rf_univdorm,
                    ceemdan_lstm_univdorm,proposed_method_univdorm_df])
result_univdorm=result_univdorm.T
result_univdorm.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
univdorm_summary=pd.concat([result_univdorm,running_time_univdorm],axis=0)

univdorm_summary.set_axis(['MAPE(%)', 'RMSE','MAE','running time (s)'], axis='index')

univdorm_summary.style.set_caption("University Dormitory Results")
index = univdorm_summary.index
index.name = "university dormitory results"
univdorm_summary

,LR,SVR,ANN,RF,LSTM,CEEMDAN RF,CEEMDAN LSTM,Proposed Method
university dormitory results,,,,,,,,
0,6.096747,6.486764,6.719073,6.153492,7.027999,3.950790,4.146746,3.495062
1,3.091409,3.386055,3.389476,3.194408,3.504771,2.004695,2.107329,1.761992
2,2.401101,2.590120,2.640636,2.440198,2.752118,1.545388,1.617382,1.362670
0,0.137205,1.217752,4.076772,1.364407,12.786276,46.890297,141.157983,132.064451


In [ ]:
#export table to png
#dfi.export(univdorm_summary,"univdorm_summary_table.png")

## Calculate percentage improvement
### Run this following cell will calculate percentage improvement and generate output used in Section 4.4 (Table 4) for University Dormitory dataset

In [62]:
pMAPE_LR_vs_Proposed_univdorm=((lr_univdorm[0]-proposed_method_univdorm[0])/lr_univdorm[0])*100
pRMSE_LR_vs_Proposed_univdorm=((lr_univdorm[1]-proposed_method_univdorm[1])/lr_univdorm[1])*100
pMAE_LR_vs_Proposed_univdorm=((lr_univdorm[2]-proposed_method_univdorm[2])/lr_univdorm[2])*100

pMAPE_SVR_vs_Proposed_univdorm=((svr_univdorm[0]-proposed_method_univdorm[0])/svr_univdorm[0])*100
pRMSE_SVR_vs_Proposed_univdorm=((svr_univdorm[1]-proposed_method_univdorm[1])/svr_univdorm[1])*100
pMAE_SVR_vs_Proposed_univdorm=((svr_univdorm[2]-proposed_method_univdorm[2])/svr_univdorm[2])*100

pMAPE_ANN_vs_Proposed_univdorm=((ann_univdorm[0]-proposed_method_univdorm[0])/ann_univdorm[0])*100
pRMSE_ANN_vs_Proposed_univdorm=((ann_univdorm[1]-proposed_method_univdorm[1])/ann_univdorm[1])*100
pMAE_ANN_vs_Proposed_univdorm=((ann_univdorm[2]-proposed_method_univdorm[2])/ann_univdorm[2])*100

pMAPE_RF_vs_Proposed_univdorm=((rf_univdorm[0]-proposed_method_univdorm[0])/rf_univdorm[0])*100
pRMSE_RF_vs_Proposed_univdorm=((rf_univdorm[1]-proposed_method_univdorm[1])/rf_univdorm[1])*100
pMAE_RF_vs_Proposed_univdorm=((rf_univdorm[2]-proposed_method_univdorm[2])/rf_univdorm[2])*100

pMAPE_LSTM_vs_Proposed_univdorm=((lstm_univdorm[0]-proposed_method_univdorm[0])/lstm_univdorm[0])*100
pRMSE_LSTM_vs_Proposed_univdorm=((lstm_univdorm[1]-proposed_method_univdorm[1])/lstm_univdorm[1])*100
pMAE_LSTM_vs_Proposed_univdorm=((lstm_univdorm[2]-proposed_method_univdorm[2])/lstm_univdorm[2])*100

pMAPE_ceemdan_rf_vs_Proposed_univdorm=((ceemdan_rf_univdorm[0]-proposed_method_univdorm[0])/ceemdan_rf_univdorm[0])*100
pRMSE_ceemdan_rf_vs_Proposed_univdorm=((ceemdan_rf_univdorm[1]-proposed_method_univdorm[1])/ceemdan_rf_univdorm[1])*100
pMAE_ceemdan_rf_vs_Proposed_univdorm=((ceemdan_rf_univdorm[2]-proposed_method_univdorm[2])/ceemdan_rf_univdorm[2])*100


pMAPE_ceemdan_lstm_vs_Proposed_univdorm=((ceemdan_lstm_univdorm[0]-proposed_method_univdorm[0])/ceemdan_lstm_univdorm[0])*100
pRMSE_ceemdan_lstm_vs_Proposed_univdorm=((ceemdan_lstm_univdorm[1]-proposed_method_univdorm[1])/ceemdan_lstm_univdorm[1])*100
pMAE_ceemdan_lstm_vs_Proposed_univdorm=((ceemdan_lstm_univdorm[2]-proposed_method_univdorm[2])/ceemdan_lstm_univdorm[2])*100


df_PI_univdorm=[[pMAPE_LR_vs_Proposed_univdorm,pMAPE_SVR_vs_Proposed_univdorm,pMAPE_ANN_vs_Proposed_univdorm,
                pMAPE_RF_vs_Proposed_univdorm,pMAPE_LSTM_vs_Proposed_univdorm,pMAPE_ceemdan_rf_vs_Proposed_univdorm,
                pMAPE_ceemdan_lstm_vs_Proposed_univdorm],
                [pRMSE_LR_vs_Proposed_univdorm,pRMSE_SVR_vs_Proposed_univdorm,pRMSE_ANN_vs_Proposed_univdorm,
                pRMSE_RF_vs_Proposed_univdorm,pRMSE_LSTM_vs_Proposed_univdorm,pRMSE_ceemdan_rf_vs_Proposed_univdorm,
                pRMSE_ceemdan_lstm_vs_Proposed_univdorm],
                [pMAE_LR_vs_Proposed_univdorm,pMAE_SVR_vs_Proposed_univdorm,pMAE_ANN_vs_Proposed_univdorm,
                pMAE_RF_vs_Proposed_univdorm,pMAE_LSTM_vs_Proposed_univdorm,pMAE_ceemdan_rf_vs_Proposed_univdorm,
                pMAE_ceemdan_lstm_vs_Proposed_univdorm]]

PI_univdorm=pd.DataFrame(df_PI_univdorm, columns=["Proposed Method vs.LR", "Proposed Method vs.SVR"," Proposed Method vs.ANN",
                                      "Proposed Method vs.RF","Proposed Method vs.LSTM","Proposed Method vs.CEEMDAN RF",
                                      "Proposed Method vs. CEEMDAN LSTM"])
PI_univdorm= PI_univdorm.round(decimals = 2)
PI_univdorm.set_axis(['MAPE(%)', 'RMSE','MAE'], axis='index')
PI_univdorm.style.set_caption("Percentage Improvement-University Dormitory Building")
index = PI_univdorm.index
index.name = "Percentage Improvement university dormitory"
PI_univdorm

,Proposed Method vs.LR,Proposed Method vs.SVR,Proposed Method vs.ANN,Proposed Method vs.RF,Proposed Method vs.LSTM,Proposed Method vs.CEEMDAN RF,Proposed Method vs. CEEMDAN LSTM
Percentage Improvement university dormitory,,,,,,,
0,42.67,46.12,47.98,43.20,50.27,11.54,15.72
1,43.00,47.96,48.02,44.84,49.73,12.11,16.39
2,43.25,47.39,48.40,44.16,50.49,11.82,15.75


In [ ]:
#export table to png
#dfi.export(PI_univdorm,"PI_univdorm_table.png")